In [46]:
import dependency as dp
import importlib
importlib.reload(dp)
from pathlib import Path

project = dp.Project(f"{str(Path.home())}/git/raox/", "ru.bmstu.rk9.rao.ui")
bundle = dp.Bundle("/Users/algor/.p2/", project.module_root, project)
bundle.update_jars()
bundle.update_dependencies()
# bundle = dp.Bundle("/Users/algor/.p2/", "org.eclipse.swt")
# bundle.update_jars()
# bundle.update_dependencies()
# bundle.dependencies

not tycho


In [49]:
exported_deps = bundle.collect_exported_dependencies()
dp.pretty_print(exported_deps)
len(exported_deps)

[
	org.eclipse.e4.ui.ide | 2 jars | 3 deps | 0 exported,
	org.eclipse.xtext.util | 2 jars | 5 deps | 3 exported,
	org.eclipse.core.filebuffers | 2 jars | 4 deps | 0 exported,
	org.eclipse.ui.ide | 2 jars | 24 deps | 1 exported,
	org.eclipse.xtext.ui.codetemplates.ui | 2 jars | 16 deps | 2 exported,
	org.eclipse.emf.edit.ui | 2 jars | 9 deps | 4 exported,
	org.eclipse.equinox.app | 2 jars | 2 deps | 0 exported,
	org.eclipse.core.resources | 2 jars | 4 deps | 0 exported,
	org.eclipse.xtext.xbase.ui | 2 jars | 27 deps | 2 exported,
	org.eclipse.core.contenttype | 2 jars | 3 deps | 0 exported,
	org.eclipse.xtext.common.types.ui | 2 jars | 17 deps | 2 exported,
	org.eclipse.emf.mwe.core | 1 jars | 5 deps | 3 exported,
	org.eclipse.emf.edit | 2 jars | 4 deps | 3 exported,
	org.eclipse.emf.ecore | 2 jars | 3 deps | 1 exported,
	com.google.guava | 2 jars | 0 deps | 0 exported,
	org.eclipse.swt | 1 jars | 6 deps | 6 exported,
	org.eclipse.ltk.core.refactoring | 2 jars | 7 deps | 0 exported,
	or

68

In [34]:
str(Path.home())

'/Users/algor'